In [1]:
import datatable as dt

# Variables that contains the file location
from files import *
from functions import *
import pandas as pd

In [2]:
# if we modify the file we need to reload it with this
import importlib
import functions  #import the module here, so that it can be reloaded.

importlib.reload(functions)

<module 'functions' from '/Users/eduardohdz/Documents/JKU/Semester1/MMSR_REPO/project/functions.py'>

In [3]:
DATA_TYPE = np.float32

# Data
Load the Data

In [4]:
relevant_results_by_id = pd.read_csv('./data/relevant_results_by_newid.csv').set_index('index').astype(np.int32)
relation_ids = pd.read_csv("./data/relation_original_new_ids.csv").set_index('original').astype(np.int32)
id_to_key = dict(zip(relation_ids.index.values, relation_ids['newId'].values))
relevant_results_by_id.index = relation_ids["newId"].sort_values().index.values

metrics_cosine_individual = get_metrics_file('./data/df_allmetrics_cosine_individual.csv',{}, _, _,_)
metrics_cosine_individual

,MAP_10,MAP_100,MRR_10,MRR_100,Mean NDCG_10,Mean NDCG_100,%DeltaMean,S10,S100
baseline,0.538468,0.468318,0.597873,0.602863,0.445806,0.445989,0.021471,0.323246,0.094932
tfidf,0.613005,0.533717,0.675660,0.679565,0.522946,0.509408,0.062432,4.687369,4.214574
word2vec,0.620418,0.531777,0.688515,0.691910,0.526673,0.505861,0.102609,16.733727,10.633337
bert,0.662683,0.582197,0.724922,0.727865,0.580295,0.558143,0.057727,12.455223,6.382306
mfcc_bow,0.686826,0.611884,0.745529,0.748227,0.609634,0.589270,0.057407,3.542412,2.684883
mfcc_stats,0.664758,0.590062,0.723587,0.726534,0.585111,0.566645,0.044250,4.471769,3.898211
essentia,0.589588,0.518582,0.651804,0.655771,0.500432,0.496907,0.019444,-0.049851,-3.908131
blf_delta_spectral,0.660296,0.580015,0.724785,0.727699,0.576541,0.556345,0.041429,5.728301,4.060359
blf_correlation,0.659284,0.576151,0.721592,0.724590,0.575718,0.550553,0.097241,6.644647,5.174368
blf_logfluc,0.684356,0.606021,0.743860,0.746583,0.606172,0.580933,0.046486,3.961619,2.879824


In [5]:
relevant_results_by_id.head(2)

,relevants
0009fFIM1eYThaPg,28841
0010xmHR6UICBOYT,2787


In [6]:
genres = dt.fread(file_genres_2).to_pandas()
genres.set_index('id', inplace=True)
# First, we change the genre column to be a list
genres['genre']= genres.genre.apply(lambda x: get_genres(x))

In [7]:
# Our special child
blf_logfluc = dt.fread(file_blf_logfluc)
blf_logfluc[dt.float64] = dt.float32
new_cols = ['id']
new_cols.extend(list(blf_logfluc.names[2:]))
new_cols = tuple(new_cols)
del blf_logfluc[:, -1]
blf_logfluc.names = new_cols
blf_logfluc = blf_logfluc.to_pandas()
blf_logfluc.set_index('id', inplace=True)

In [8]:
files = [
    # Lyrics
    file_tfidf_2,
#     file_word2vec_2,
    file_bert_2,
    # Audio
    # file_essentia, # zeros
#     file_blf_correlation,
#     file_blf_deltaspectral,
    file_blf_spectral,
#     file_blf_spectralcontrast,
#     file_blf_vardeltaspectral,
#     file_blf_logfluc, # zeros
    blf_logfluc,
    file_mfcc_bow,
#     file_mfcc_stats,
    # Video
    file_incp,
    file_resnet,
#     file_vgg19,
]

In [9]:
functions = [get_cosine_similarity]

In [10]:
common_index = blf_logfluc.index.values

In [11]:
import csv

def load_data(file):
    if type(file) == str:
        with open(file, "r") as f:
            column_names = next(csv.reader(f, delimiter="\t"))
        dtypes = {
            x: np.float32
            for x in column_names
            if x != "id"
        }
        file = pd.read_csv(file, dtype=dtypes, index_col=0, delimiter="\t")
    file = file.loc[common_index]
    assert np.mean(file.index.values == blf_logfluc.index.values) == 1.0
    return file

In [12]:
# Extract readable names
names = []
for file in files:
    if type(file) == str:
        names.append(file.replace("./../task2/id_", "").replace(".tsv", ""))
    else:
        names.append("blf_logfluc")

# Late Fusion

In [13]:
def compute_in_batches_topIds(results: np.array, idx_values: np.array, top: int = 100, batches: int = 1):
    splits_b = np.array_split(results, batches, axis=0)
    return np.concatenate([idx_values[np.argsort(b * -1, axis=1)][:, :top] for b in tqdm(splits_b)], axis=0)

# Find optimal feature similarity weights

In [14]:
subset = np.random.choice(len(blf_logfluc), 1024 * 4, replace=False)
subset.shape

(4096,)

In [15]:
subset_ids = genres.index.values[subset]

similarities = []

# for every data
i = 0
for d in tqdm(files, desc="Processing"):
    file = load_data(d).to_numpy()[subset, :].astype(np.float32)
    for f in functions:
        result = np.nan_to_num(f(file, file), copy=False)
        similarities.append(result)
    i += 1

print(f"{len(similarities)} similarity matrices")

Processing: 100%|█████████████████████████████████| 7/7 [02:07<00:00, 18.17s/it]

7 similarity matrices


In [16]:
similarities[0].shape

(4096, 4096)

In [17]:
def evaluate(similarities, weights):
    if len(similarities) == 1:
        accumulated_similarity = similarities[0]
    else:
        accumulated_similarity = np.zeros((len(subset), len(subset)))
        for sim, weight in zip(similarities, weights):
            accumulated_similarity += (sim * weight)

    top_ids = pd.DataFrame(compute_in_batches_topIds(accumulated_similarity, subset_ids, 100, 10), subset_ids)
    
    map10, mrr10, ndcg10 = getMetrics(top_ids, 10, genres, relevant_results_by_id)
    return {
        "MAP@10": map10,
        "MRR@10": mrr10,
        "NDCG@10": ndcg10,
    }

In [18]:
print("Baseline", evaluate(similarities, np.zeros((len(similarities),))))

100%|█████████████████████████████████████| 4096/4096 [00:03<00:00, 1077.18it/s]

Baseline {'MAP@10': 0.520131360258495, 'MRR@10': 0.6203880673363096, 'NDCG@10': 0.414009972032934}


In [19]:
print("Fair", evaluate(similarities, np.ones((len(similarities),))))

100%|█████████████████████████████████████| 4096/4096 [00:03<00:00, 1064.97it/s]

Fair {'MAP@10': 0.8072114897459092, 'MRR@10': 1.0, 'NDCG@10': 0.5599119154594183}


In [20]:
weights = np.zeros((len(similarities),))
for i, sim in enumerate(similarities):
    result = np.nan_to_num(np.asarray(list(evaluate([sim], [1]).values())), copy=False)
    print(result)
    weights[i] = np.mean(result, where=result > 0)
print(weights)

100%|█████████████████████████████████████| 4096/4096 [00:03<00:00, 1067.82it/s]


[0.79772443 0.99987793 0.55034057]


100%|█████████████████████████████████████| 4096/4096 [00:03<00:00, 1094.26it/s]


[0.79703637 0.99987793 0.54222581]


100%|█████████████████████████████████████| 4096/4096 [00:03<00:00, 1082.58it/s]


...

[0.80727532 1.         0.56099649]


100%|█████████████████████████████████████| 4096/4096 [00:03<00:00, 1028.82it/s]


[0.801461   1.         0.55066502]


100%|█████████████████████████████████████| 4096/4096 [00:03<00:00, 1048.62it/s]

[0.79887175 1.         0.54834659]
[0.78264764 0.77971337 0.78721394 0.78932727 0.78942394 0.784042
 0.78240611]


In [21]:
features_considered = ["tfidf", "bert", "blf_spectral", "blf_logfluc", "mfcc_bow", "incp", "resnet"]
metrics_considered_10 = ["MAP_10", "MRR_10", "Mean NDCG_10"]
metrics_considered_100 = ["MAP_100", "MRR_100", "Mean NDCG_100"]
weights_10 = metrics_cosine_individual.loc[features_considered,metrics_considered_10].mean(axis=1).values
weights_100 = metrics_cosine_individual.loc[features_considered,metrics_considered_100].mean(axis=1).values
print("Weights considering @10 metrics:\n",weights_10, "\nWeights considering @100 metrics:\n",weights_100)

Weights considering @10 metrics:
 [0.60387002 0.65596642 0.6858311  0.67812915 0.68066308 0.69186326
 0.69831527] 
Weights considering @100 metrics:
 [0.57422996 0.62273479 0.65315864 0.64451217 0.64979362 0.63357245
 0.63816635]


In [22]:
print("Performance", evaluate(similarities, weights))

100%|█████████████████████████████████████| 4096/4096 [00:03<00:00, 1043.05it/s]

Performance {'MAP@10': 0.8075367407343435, 'MRR@10': 1.0, 'NDCG@10': 0.5598964999400058}


In [23]:
norm_weights = (weights - weights.min()) / (weights.max() - weights.min())
print(norm_weights)
print("Normalized Performance", evaluate(similarities, norm_weights))

[0.3021731  0.         0.77241359 0.9900448  1.         0.44576546
 0.27730047]


100%|█████████████████████████████████████| 4096/4096 [00:04<00:00, 1021.72it/s]

Normalized Performance {'MAP@10': 0.8078072121702594, 'MRR@10': 1.0, 'NDCG@10': 0.5635511756754903}


In [24]:
norm_weights_10  = (weights_10  - weights_10.min())  / (weights_10.max()  - weights_10.min())
norm_weights_100 = (weights_100 - weights_100.min()) / (weights_100.max() - weights_100.min())
print("Normalized Weights considering @10 metrics:\n",
      norm_weights_10, 
      "\nNormalized Weights considering @100 metrics:\n",
      norm_weights_100)
print("Normalized Performance @10", evaluate(similarities, norm_weights_10))
print("Normalized Performance @100", evaluate(similarities, norm_weights_100))

Normalized Weights considering @10 metrics:
 [0.         0.55160421 0.86781581 0.78626641 0.81309598 0.93168511
 1.        ] 
Normalized Weights considering @100 metrics:
 [0.         0.61453991 1.         0.89045204 0.95736627 0.75184954
 0.81005272]


100%|█████████████████████████████████████| 4096/4096 [00:03<00:00, 1083.99it/s]


Normalized Performance {'MAP@10': 0.8050223639102089, 'MRR@10': 1.0, 'NDCG@10': 0.5565558829068067}


100%|█████████████████████████████████████| 4096/4096 [00:03<00:00, 1084.92it/s]

Normalized Performance {'MAP@10': 0.8048238972735723, 'MRR@10': 1.0, 'NDCG@10': 0.5583612006332563}


In [25]:
del similarities

# Final Processing
Normalized Performance seems to be the best approach

In [119]:
final_ids = blf_logfluc.index.values
final_accumulated_similarity = np.zeros((len(final_ids), len(final_ids)), dtype=DATA_TYPE)

In [120]:
# for every data
batches = 100
i = 0
for d in files:
    file = load_data(d).to_numpy()
    for f in functions:
        splits = np.array_split(file, batches, axis=0)
        y = 0
        for b in tqdm(splits):
            result = f(file, b).astype(np.float32)

            result = np.nan_to_num(result, copy=False)

            # normalize
            np.subtract(result, result.mean(), out=result)

            # normalize std and apply weight
            result = np.multiply(result, norm_weights_100[i] / result[::64, ::64].std(), out=result)

            final_accumulated_similarity[:, y:y + b.shape[0]] += result
            y += b.shape[0]
        print("Sanity Check", final_accumulated_similarity[:1024, :1024].mean())
        i += 1

100%|█████████████████████████████████████████| 100/100 [01:22<00:00,  1.21it/s]


Sanity Check 0.0


100%|█████████████████████████████████████████| 100/100 [01:08<00:00,  1.45it/s]


Sanity Check 0.007916106


100%|█████████████████████████████████████████| 100/100 [01:16<00:00,  1.31it/s]


...

Sanity Check -0.053062297


100%|█████████████████████████████████████████| 100/100 [03:31<00:00,  2.11s/it]


Sanity Check -0.08761212


100%|█████████████████████████████████████████| 100/100 [03:27<00:00,  2.07s/it]

Sanity Check -0.109552905


In [121]:
# replace diagonals by -inf to prevent them being picked
np.fill_diagonal(final_accumulated_similarity, -np.inf)

In [122]:
np.save("final_similarity.p", final_accumulated_similarity)

In [123]:
top_ids = pd.DataFrame(compute_in_batches_topIds(final_accumulated_similarity, final_ids, 100, 100), final_ids)

100%|█████████████████████████████████████████| 100/100 [07:08<00:00,  4.28s/it]


In [124]:
dt.Frame(pd.DataFrame(top_ids, index=final_ids).reset_index()).to_csv('./top_ids_late_fusion_test.csv')

In [125]:
top_ids

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0009fFIM1eYThaPg,PHYuZDQa15QV9Rlc,Uly30J1KEdC9awfI,aLdUurk83dJm8w7W,wLrYaZGgflCudkS2,yLDYDyhZ3FFmc2Ys,fyQyjwU2vsgFEl1r,dw8IkK4rPlDamYsd,U3Zo3ToDKW72HNlb,UMahojluDMtosnva,CY2PCIr7j680lCzs,...,KQNFVVsibe52n7lE,ZOTTT1zf1urrHMV2,Rh4Bn2msKgz7m3YE,TPpooNuXSy6lsRip,wo3249plMXd8cM0R,1eunHqu243peeQtE,HIon1AMiKfDZzk9K,ZIBXdQ5DxrjQhzhH,0LAm6RiCVmW0kxld,GAeGNMTGrp6ky2um
0010xmHR6UICBOYT,tvsZaTDnNqZPZiQC,zp3KKKwIyeSDbrkf,MVdLRZiR3M35J4b0,SSogs1SYlokISgEq,Awwf1OBE6Q6vbyum,A2Sjds83oq6NW2D4,7JVOiEwXe5SwTgzT,TQ6fkPLslfSQORvf,9w9gcgpKBBcZMXHL,L2TXDgiHNytj8HRk,...,mfwyHt6r4y7cCgnP,xjqBYyCebMEDc7Id,xjPbsW9OgACGpbD5,LKyKOXUr3NkTOYNe,TcClCCLPQHTLN5BX,phrRAxDxoOmOm2VH,vPzYDzVCnl1K1ihn,JPOLE03VfYw9GZyc,n7qIV7kovU9P5OIo,dmuMejFkWiSKKHIs
002Jyd0vN4HyCpqL,2S9Tq0XzZ7iAovKv,wQyodz8psUzd3447,2ByaOIPZJ7irmkKG,lwjCqRX8vexafTOd,foVvryczNWG8CoFR,RTVljF6OFjepeQgq,R73XiLEKVA7gs36b,1OM5El3UXnKVVTtN,7QDB0NEIsUDbZWSP,TrUXW5OQ9OaNgKmX,...,eKFLMuJWGfeCig5p,Rxx3qPyrMMyfvB7V,FqoLwE9pXVu1kSb6,Vt0evHg09P5QY0Vc,gJDG0Azt0Oe7qIKZ,fMxtMdUVUooRnFv3,YHupaKUf3ndWCU0v,lS0plxOmG29oGgJg,2Utl2NhhjRa2p5Xu,ET5DOul1I2cnXil0
006TYKNjNxWjfKjy,0oe2d23jio9Uyg49,U3KiFNPvWCXhOuPm,zI3j3T5qkiylIBaD,wR4F3A90kVplf4bJ,UceDke4Q33GCS6nY,LbKaKfHDu7qbcF45,q0H90swl2jbhmN33,7xXPEA7GTgetirvj,XAV6f5jnhKNBZ5Uv,lVxQcb44bWLXjIKS,...,7QBaCghqJ4kN9Xju,I1xe2qFofypXrh3e,9AReP2499tVpr8zf,O4Jos6slSYWyJ1KT,gilVuAjVhRUYDyal,Y8uKE3rK9JpiiMJx,chkKFoRchCEWp04Z,IZG8JFzdF8qY24XU,lqT19WfqMtplTXha,6jTNLP3p3mpmEJyL
007LIJOPQ4Sb98qV,cyRMqYRBDcRleNsa,qsmfGwfCd4FFaDet,wOhO1MgW1CcEKaAx,S1rroi8TiRqPNWDX,ljHcf8k4kcRQ525P,053TqfoRuva4bHoj,byCM04NPd9bD0SEW,0Hhuy13DlIS8loEY,t5NHA3VKvjlBxk8Y,atbVc1L7CNJTbM14,...,q3QjZmXncdv89Bu3,GDqJrwcTkgK7NQJ7,CcU6Z5YxEcuSeBYM,ixdMVnnFtQb9d6fA,uERrHBJajOBQM3r2,a2bC8PID2eQzeNFn,BX7jubmUk9FgMxmO,QdXlbqjwbM6KICSb,uBYnZ4kGBmdpEPmh,If0UmHl6wDMm6pxZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzx8CWdM7qkxKQpC,pF2UjM0abOPwWfBm,1WmILK0emIsdTSl2,CGz0z5iv0jZmIa7X,jBflH3Pbnmu6w9PN,ZE8BGOMLs4haWgrz,Ix3E8QzXaW2l60z0,UEH28vtjfDBIbDRd,ZCmYDEN9oVJu03HY,aRdWAZwTojMjsxbN,jUt92xv9HKrFID8G,...,dcyOoMBVWuoptXdn,Nm2A0A2FojNYTKiQ,K0UTWVBEh7A8hUAM,9fb5miQdgj3NV720,7WCzT2gCk8Qq50AH,7nrWxLXgH9oTDozL,Bo8FpTTaGWX20eZX,UrtAuKMvJaYEhJHy,RiYyZ7GvntHmjG1L,wLsmR9w5wTbaL6AI
zzyb5LvKJTWLVnrk,0Pn0EYWzup45hJB5,D5z0dWO50sLyTDWE,3bejCrhNQlm97r9A,qaeZAwbRSlV5IB4o,1c0qoBueLYugXqQq,nCpr1adAFNm7flwN,W89wMsilBSmysiw4,SPnjOznLAalAxf30,A0THNQJasLtFoBBj,0UF21Ipg9OUQh3Rs,...,pF5dqrJwRreBWon6,2ZFMLcQ9rWNndcLi,JZO5uFJP3uUonss4,BrDKQo2gQvStSlEp,P8Cy3Ax7KtsCrIPU,oHzidGPUYlkSnAO1,dC403UAHdfIblZ3a,CGZyuzZLcwkoiUW5,umLVO58k5TGDNO7W,nvyoRFZVyjBwBMlS
zzz0n04uuTUA7fNh,ldBb9ph0mF5speiz,qHfZgrnidIOsiZph,U003NP8GAkh58HKP,9gFGFPGnvnOgKNCp,SMnpUHHEhMdLJqJ2,ZJEmQhq6tEm0FFfA,uRNge3sN74NdIaYi,8sufJfhyteY1VXr7,ZQiK2a39fHk4AUuy,2IZjV6srFH5YXkFr,...,czlMQGKVuOTrL0OW,4S8s0vwC5BSH2Npt,h38AVryexbGR1hOY,FjpuaUU3m7bP27e8,2G3Vke7a7vBxZ0qc,9gdfbpxc6bWHZk8j,fGzmpsiijB9s2dGF,mpd8nWUscKpDz9Yt,aFh2EhTk1dt8MwV6,XmbmtmCRZ9c6mVUf
zzznMjZAKnJJXQSj,m3bU7wEiG8i3QgLU,XSV2iTgL6MBxHuSg,ZkZYGam1rh4ofsot,9EblKA932h5UjT4o,UGqLXvmfFV1mccyr,arJV2x5YxfzhWhHT,igPQmB6xcoLu5kvm,ZyYqsjDXfsUzGUOG,Qr5OyRAwsoWpAUCZ,7PCTOXUwR2lO6jQy,...,1Gp0KRB1Uge4r8EX,Hq6PYAuJnjnVleH7,L8D6pJk1t29Q12JI,JP07B6JyIek40VBe,SgBvRaxOi7XaDEmU,7QgcMKJoKBlXzX2n,TyR2s4rEgr4kpajt,BqN9q6xrn5AmmXE5,6egVjrcc60M43i5N,7alHkTpqV2cbf8G6


In [126]:
map10, mrr10, ndcg10 = getMetrics(top_ids, 10, genres, relevant_results_by_id)
map100, mrr100, ndcg100 = getMetrics(top_ids, 100, genres, relevant_results_by_id)
results ={
    "MAP@10": map10,
    "MAP@100": map100,
    "MRR@10": mrr10,
    "MRR@100": mrr100,
    "NDCG@10": ndcg10,
    "NDCG@100": ndcg100,
}

100%|████████████████████████████████████| 68641/68641 [01:24<00:00, 814.71it/s]


In [127]:
results

{'MAP@10': 0.7569790013997189,
 'MAP@100': 0.6636791432633635,
 'MRR@10': 0.8158885961303614,
 'MRR@100': 0.8178115813326634,
 'NDCG@10': 0.681603814931831,
 'NDCG@100': 0.6371542878933193}

In [79]:
# considering samples with @10
results

{'MAP@10': 0.7577162847911041,
 'MAP@100': 0.6589291696147251,
 'MRR@10': 0.8179865601636117,
 'MRR@100': 0.8199689688761987,
 'NDCG@10': 0.680669241568677,
 'NDCG@100': 0.6310221049110392}

In [32]:
# considering samples with @100
results

{'MAP@10': 0.7473254334734364,
 'MAP@100': 0.6486563802524675,
 'MRR@10': 0.8093814192683672,
 'MRR@100': 0.8114731162664771,
 'NDCG@10': 0.6685461241485902,
 'NDCG@100': 0.6203359765292428}

In [109]:
# considering whole metrics with @10
results

{'MAP@10': 0.7612543309394363,
 'MAP@100': 0.6635917544366515,
 'MRR@10': 0.8205788721766781,
 'MRR@100': 0.822516142818151,
 'NDCG@10': 0.6846565677583085,
 'NDCG@100': 0.6358935658803938}

In [128]:
# considering whole metrics with @100
results

{'MAP@10': 0.7569790013997189,
 'MAP@100': 0.6636791432633635,
 'MRR@10': 0.8158885961303614,
 'MRR@100': 0.8178115813326634,
 'NDCG@10': 0.681603814931831,
 'NDCG@100': 0.6371542878933193}